In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms, utils
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

In [2]:
import os

os.chdir('../../experiments')

In [3]:
directory = "../data"

In [4]:
def show_images_from_loader(dataloader, num_images=5):
    data_iter = iter(dataloader)  # Get an iterator over DataLoader
    images, labels = next(data_iter)  # Get the first batch

    fig, axes = plt.subplots(1, num_images, figsize=(12, 4))
    for i in range(num_images):
        image = images[i].squeeze(0).permute(1, 2, 0).numpy()  # Convert tensor (C, H, W) to (H, W, C)
        axes[i].imshow(image)
        axes[i].set_title(f"Class {labels[i].item()}")
        axes[i].axis("off")
    
    plt.show()

In [5]:
def generate_random_color_set(length):
   
    # Generate random integers in the range [0, 255] for each color channel
    color_set = np.random.randint(0, 256, size=(length, 3))
    return color_set / 255.0 # Normalize to [0, 1]


#### Transformer Functions

In [21]:
def colorize_transform(img, color_set):
    # Convert tensor to numpy and remove channel dimension added by ToTensor()
    img = np.array(img).squeeze(0)  # Now shape (28, 28)
    
    # Normalize and expand to (28, 28, 1)
    img = img / 255.0
    img = np.expand_dims(img, axis=-1)  # Shape: (28, 28, 1)
    
    # Map to colors
    num_colors = len(color_set)
    color_indices = (img * (num_colors - 1)).astype(int)
    colorized_img = color_set[color_indices]  # Shape: (28, 28, 1, 3)
    
    # Remove the unnecessary singleton dimension
    colorized_img = colorized_img.squeeze(2)  # Shape: (28, 28, 3)
    
    # Convert to tensor and permute to (C, H, W)
    return torch.tensor(colorized_img, dtype=torch.float32).permute(2, 0, 1)

#### Train Dataset Instances

[don't run this cell after you move on and run to the next cells]

[we want to preserve the random color set just produced]

[to colorize the test set with it too]

In [22]:
color_numbers = [2, 4, 8, 16, 32]
COLOR_SET = [color_set for color_set in map(generate_random_color_set, color_numbers)]

In [23]:
colorized_transforms = []
for i in range(len(COLOR_SET)):
    colorized_transforms.append(
        transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor first
    transforms.Lambda(lambda x: colorize_transform(x * 255, COLOR_SET[i]))  # Apply colorization
])
    )

In [24]:
train_datasets = []
for transform in colorized_transforms:
    train_datasets.append(datasets.FashionMNIST(directory, train=True, download=True, transform=transform))

In [25]:
train_loaders = []
for dataset in train_datasets:
    train_loaders.append(DataLoader(dataset, batch_size=64, shuffle=True))

### Training

In [26]:
from train import train, test
from cnn import _3LayerCNN

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##### Random Initialization I

In [ ]:
models_r1 = [ _3LayerCNN().to(device) for i in range(len(train_loaders))]
optimizers = [optim.Adam(model.parameters()) for model in models_r1]

In [ ]:
for i in range(len(models_r1)):
    train(models_r1[i], device, train_loaders[i], optimizers[i], epoch=10)
    

##### Random Initialization II

In [ ]:
models_r2 = [ _3LayerCNN().to(device) for i in range(len(train_loaders))]
optimizers = [optim.Adam(model.parameters()) for model in models_r2]

In [ ]:
for i in range(len(models_r2)):
    train(models_r2[i], device, train_loaders[i], optimizers[i], epoch=10)
    

##### Same Initialization I

In [60]:
def create_models(
    num_models: int,
    model_class: torch.nn.Module,
    device: torch.device,
    seed: int = 42,
    deterministic: bool = False
) -> list[torch.nn.Module]:
    """
    Create multiple models with identical weight initializations.
    
    Args:
        num_models: Number of models to create
        model_class: Model class constructor (e.g., _3LayerCNN)
        device: Target device (e.g., 'cuda' or 'cpu')
        seed: Random seed for reproducibility
        deterministic: Enable full reproducibility (slower performance)
    
    Returns:
        List of initialized models with identical weights
    """
    # Set reproducibility flags
    torch.manual_seed(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

     # Create prototype and capture state
    prototype = model_class().to(device)
    state_dict = prototype.state_dict()

    # Create models with identical weights
    models = []
    for _ in range(num_models):
        model = model_class().to(device)
        model.load_state_dict(state_dict)  # In-place operation
        models.append(model)  # Append the model, not load_state_dict's return
    
    return models

In [61]:
# 1. Create a seed for reproducibility
SEED1 = 42
SEED2 = 144

In [62]:
models_s1 = create_models(
    num_models=len(train_loaders),
    model_class=_3LayerCNN,
    device=device,
    seed=SEED1,
    deterministic=True
)

optimizers = [optim.Adam(model.parameters()) for model in models_s1]

In [63]:
for i in range(len(models_s1)):
    train(models_s1[i], device, train_loaders[i], optimizers[i], epoch=10)

/tmp/ipykernel_92273/1492103977.py:3: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  img = np.array(img).squeeze(0)  # Now shape (28, 28)


Train Epoch: 10 [0/60000 (0%)]	Loss: 2.318068


KeyboardInterrupt: 

In [64]:
models_s2 = create_models(
    num_models=len(train_loaders),
    model_class=_3LayerCNN,
    device=device,
    seed=SEED2,
    deterministic=True
)

optimizers = [optim.Adam(model.parameters()) for model in models_s2]

In [65]:
for i in range(len(models_s2)):
    train(models_s2[i], device, train_loaders[i], optimizers[i], epoch=10)

/tmp/ipykernel_92273/1492103977.py:3: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  img = np.array(img).squeeze(0)  # Now shape (28, 28)


Train Epoch: 10 [0/60000 (0%)]	Loss: 2.299165


KeyboardInterrupt: 

#### Test Datasets Transformer functions

##### GrayScale

In [31]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to 3-channel RGB
    transforms.RandomRotation(degrees=15),  # Slight rotation for data augmentation
    transforms.ToTensor(),  # Convert to tensor
])

In [32]:
test_dataset = datasets.FashionMNIST(
    directory, train=False, transform=transform)

test_loader = DataLoader(
    test_dataset, batch_size=128, shuffle=False)

In [ ]:
for model in models_r1:
    test(model, device, test_loader)

In [ ]:
for model in models_r2:
    test(model, device, test_loader)

In [ ]:
for model in models_s1:
    test(model, device, test_loader)

In [ ]:
for model in models_s2:
    test(model, device, test_loader)

##### Baseline

In [35]:
test_datasets = []
for transform in colorized_transforms:
    test_datasets.append(datasets.FashionMNIST(directory, train=False, transform=transform))

test_loaders = []
for dataset in test_datasets:
    test_loaders.append(DataLoader(dataset, batch_size=128, shuffle=False))

In [ ]:
for i in range(len(models_r1)):
    test(models_r1[i], device, test_loaders[i])

/tmp/ipykernel_92273/1492103977.py:3: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  img = np.array(img).squeeze(0)  # Now shape (28, 28)



Test set: Average loss: 0.0177, Accuracy: 1475/10000 (14.75%)



KeyboardInterrupt: 

In [ ]:
for i in range(len(models_r2)):
    test(models_r2[i], device, test_loaders[i])

In [ ]:
for i in range(len(models_s1)):
    test(models_s1[i], device, test_loaders[i])

In [ ]:
for i in range(len(models_s2)):
    test(models_s2[i], device, test_loaders[i])

##### Same number of Color but random

In [38]:
color_numbers = [2, 4, 8, 16, 32]
COLOR_SET = [color_set for color_set in map(generate_random_color_set, color_numbers)]

In [39]:
colorized_transforms = []
for i in range(len(COLOR_SET)):
    colorized_transforms.append(
        transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor first
    transforms.Lambda(lambda x: colorize_transform(x * 255, COLOR_SET[i]))  # Apply colorization
])
    )

In [40]:
test_datasets = []
for transform in colorized_transforms:
    test_datasets.append(datasets.FashionMNIST(directory, train=False, transform=transform))

test_loaders = []
for dataset in test_datasets:
    test_loaders.append(DataLoader(dataset, batch_size=128, shuffle=False))

In [ ]:
for i in range(len(models_r1)):
    test(models_r1[i], device, test_loaders[i])

/tmp/ipykernel_92273/1492103977.py:3: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  img = np.array(img).squeeze(0)  # Now shape (28, 28)


KeyboardInterrupt: 

In [ ]:
for i in range(len(models_r2)):
    test(models_r2[i], device, test_loaders[i])

In [ ]:
for i in range(len(models_s1)):
    test(models_s1[i], device, test_loaders[i])

In [ ]:
for i in range(len(models_s2)):
    test(models_s2[i], device, test_loaders[i])

##### 32 Number of Colors

In [48]:
random_32_colors  = generate_random_color_set(32)
type(random_32_colors)

numpy.ndarray

In [44]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor first
    transforms.Lambda(lambda x: colorize_transform(x * 255, random_32_colors))  # Apply colorization
])

In [45]:
test_dataset = datasets.FashionMNIST(
    directory, train=False, transform=transform)

test_loader = DataLoader(
    test_dataset, batch_size=128, shuffle=False)

In [ ]:
for model in models_r1:
    test(model, device, test_loader)

/tmp/ipykernel_92273/1492103977.py:3: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  img = np.array(img).squeeze(0)  # Now shape (28, 28)


KeyboardInterrupt: 

In [ ]:
for model in models_r2:
    test(model, device, test_loader)

In [ ]:
for model in models_s1:
    test(model, device, test_loader)

In [ ]:
for model in models_s2:
    test(model, device, test_loader)